데이터 설명

train/test는 14개의 columns으로 구성되어 있고, train은 예측해야 하는 target 값 feature까지 1개가 추가로 있습니다. 각 데이터는 다음을 의미합니다.

id
age : 나이
workclass : 고용 형태
fnlwgt : 사람 대표성을 나타내는 가중치 (final weight의 약자)
education : 교육 수준
education_num : 교육 수준 수치
marital_status: 결혼 상태
occupation : 업종
relationship : 가족 관계
race : 인종
sex : 성별
capital_gain : 양도 소득
capital_loss : 양도 손실
hours_per_week : 주당 근무 시간
native_country : 국적
income : 수익 (예측해야 하는 값)
>50K : 1
<=50K : 0

데이터 불러오기

In [1]:
import warnings

warnings.filterwarnings(action='ignore')

from os.path import join
import pandas as pd
import numpy  as np
import xgboost as xgb


sample_data = pd.read_csv(join('data', join('kaggle', 'train.csv')))

In [2]:
sample_data

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,0,40,Private,168538,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,60,United-States,>50K
1,1,17,Private,101626,9th,5,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,20,United-States,<=50K
2,2,18,Private,353358,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,16,United-States,<=50K
3,3,21,Private,151158,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,25,United-States,<=50K
4,4,24,Private,122234,Some-college,10,Never-married,Adm-clerical,Not-in-family,Black,Female,0,0,20,?,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26044,26044,57,Private,250201,11th,7,Married-civ-spouse,Other-service,Husband,White,Male,0,0,52,United-States,<=50K
26045,26045,23,Private,238092,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K
26046,26046,78,?,165694,Masters,14,Widowed,?,Not-in-family,White,Female,0,0,15,United-States,<=50K
26047,26047,26,Self-emp-not-inc,151626,HS-grad,9,Never-married,Prof-specialty,Own-child,Black,Female,0,0,40,United-States,<=50K


파생변수 생성
capital  = capital_gain - capital_loss

age_group 설정

country_group 설정

education_group 설정

In [3]:
#파생변수 생성(capital)
sample_data['capital'] = sample_data['capital_gain'] - sample_data['capital_loss']

In [4]:
sample_data.drop(columns=['capital_gain','capital_loss'],inplace=True)

In [5]:
#파생변수 생성(age_group)
bins = [0, 29, 39, 64, 120]
labels = ['youth', 'middle', 'middle_elder', 'elder']
sample_data['age_group'] = pd.cut(sample_data.age, bins, labels = labels,include_lowest = True)

In [6]:
sample_data.drop(columns=['age'],inplace=True)

In [7]:
#파생변수 생성(country_group)
condition = [
    sample_data['native_country'].str.contains('United-States|Canada', na=False),
    sample_data['native_country'].str.contains('Mexico|Cuba|El-Salvador|Guatemala|Columbia|Peru|Nicaragua|Honduras|Ecuador', na=False),
    sample_data['native_country'].str.contains('Poland|Germany|France|Ireland|England|Italy|Scotland|Holand-Netherlands', na=False),
    sample_data['native_country'].str.contains('Portugal|Hungary|Greece|Yugoslavia', na=False),
    sample_data['native_country'].str.contains('Philippines|Vietnam|Thailand|Laos|Cambodia', na=False),
    sample_data['native_country'].str.contains('Taiwan|Japan|China|Hong|South', na=False),
    sample_data['native_country'].str.contains('Iran|India', na=False),
    sample_data['native_country'].str.contains('Puerto-Rico|Haiti|Trinadad&Tobago|Outlying-US(Guam-USVI-etc)', na=False),
    sample_data['native_country'].str.contains('Jamaica|Dominican-Republic', na=False)
]

choices = ['N_America', 'S_America', 'High_Europe', 'Mid_Europe', 'ES_Asia', 'E_Asia', 'W_Asia', 'Used_Colony', 'Africa']

sample_data['country_group'] = np.select(condition, choices, default = 'United-States')

In [8]:
sample_data.drop(columns=['native_country'], inplace=True)

In [9]:
#education 변수
sample_data['education'].unique()

array(['HS-grad', '9th', 'Some-college', 'Bachelors', '11th', '1st-4th',
       '5th-6th', 'Assoc-acdm', '10th', 'Prof-school', '7th-8th',
       'Masters', 'Assoc-voc', 'Doctorate', '12th', 'Preschool'],
      dtype=object)

In [10]:
condition = [
    sample_data['education'].str.contains('Preschool',na=False),
    sample_data['education'].str.contains('1st-4th|5th-6th',na=False),
    sample_data['education'].str.contains('7th-8th|9th',na=False),
    sample_data['education'].str.contains('10th|11th|12th',na=False),
    sample_data['education'].str.contains('HS-grad',na=False),
    sample_data['education'].str.contains('Some-college',na=False),
    sample_data['education'].str.contains('Assoc-acdm|Assoc-voc',na=False),
    sample_data['education'].str.contains('Bachelors',na=False),
    sample_data['education'].str.contains('Masters',na=False),
    sample_data['education'].str.contains('Prof-school|Doctorate',na=False)
]
choices = ['pre','elementry','middleschool','highschool','highschool_grad','college','assoc','bachelors','ms','ph.d']
sample_data['education_level'] = np.select(condition, choices, default = 'others')

In [11]:
sample_data.drop(columns=['education','education_num'],inplace=True)

데이터 확인

In [12]:
sample_data

,id,workclass,fnlwgt,marital_status,occupation,relationship,race,sex,hours_per_week,income,capital,age_group,country_group,education_level
0,0,Private,168538,Married-civ-spouse,Sales,Husband,White,Male,60,>50K,0,middle_elder,N_America,highschool_grad
1,1,Private,101626,Never-married,Machine-op-inspct,Own-child,White,Male,20,<=50K,0,youth,N_America,middleschool
2,2,Private,353358,Never-married,Other-service,Own-child,White,Male,16,<=50K,0,youth,N_America,college
3,3,Private,151158,Never-married,Prof-specialty,Own-child,White,Female,25,<=50K,0,youth,N_America,college
4,4,Private,122234,Never-married,Adm-clerical,Not-in-family,Black,Female,20,<=50K,0,youth,United-States,college
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26044,26044,Private,250201,Married-civ-spouse,Other-service,Husband,White,Male,52,<=50K,0,middle_elder,N_America,highschool
26045,26045,Private,238092,Never-married,Prof-specialty,Own-child,White,Male,40,<=50K,0,youth,N_America,bachelors
26046,26046,?,165694,Widowed,?,Not-in-family,White,Female,15,<=50K,0,elder,N_America,ms
26047,26047,Self-emp-not-inc,151626,Never-married,Prof-specialty,Own-child,Black,Female,40,<=50K,0,youth,N_America,highschool_grad


#미리 제거하거나 분리해야할 변수 설정
(income의 경우 target, id 경우 필요없는 변수, workclass,occupation의 경우 따로 예측할
변수이므로)

In [13]:
label = sample_data['income']
workclass = sample_data['workclass']
occupation = sample_data['occupation']

In [14]:
sample_data.drop(columns=['id','workclass','occupation','income'], inplace=True)

연속형, 범주형 변수 구분

In [15]:
cat_columns = [c for (c, t) in zip(sample_data.dtypes.index, sample_data.dtypes) if t == 'O'] 
num_columns = [c for c in sample_data.columns if c not in cat_columns]

In [16]:
sample_data['age_group'] = sample_data['age_group'].astype('object')

In [17]:
cat_columns = [c for (c, t) in zip(sample_data.dtypes.index, sample_data.dtypes) if t == 'O'] 
num_columns = [c for c in sample_data.columns if c not in cat_columns]

In [18]:
sample_data_num = sample_data[num_columns]

In [19]:
sample_data_cat = sample_data[cat_columns]

범주형 변수의 one-hot encoding 실시

In [20]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
sample_data_cat = ohe.fit_transform(sample_data_cat)

In [21]:
ohe_columns=ohe.categories_[0].tolist()
for i in range(len(ohe.categories_)-1):
        ohe_columns += ohe.categories_[i+1].tolist()

In [22]:
sample_data_cat = pd.DataFrame(sample_data_cat,columns = ohe_columns)

연속형, 범주형 분리한 것 합치기
income, occupation 붙이기(결측치에 대해 예측하여야 하므로 따로 분리 후 합침)

In [23]:
sample_data_num.reset_index(drop=True)
sample_data_cat.reset_index(drop=True)
sample_data1 = pd.concat([sample_data_num,sample_data_cat],axis=1)

In [24]:
sample_data1.reset_index(drop=True)
label = label.reset_index(drop=True)
sample_data1 = pd.concat([sample_data1,label],axis=1)

In [25]:
sample_data1.reset_index(drop=True)
occupation = occupation.reset_index(drop=True)
sample_data1 = pd.concat([sample_data1,occupation],axis=1)

In [26]:
sample_data1

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,income,occupation
0,168538,60,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,>50K,Sales
1,101626,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,<=50K,Machine-op-inspct
2,353358,16,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K,Other-service
3,151158,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K,Prof-specialty
4,122234,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K,Adm-clerical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26044,250201,52,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,<=50K,Other-service
26045,238092,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K,Prof-specialty
26046,165694,15,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,<=50K,?
26047,151626,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,<=50K,Prof-specialty


workclass 붙이기(occupation과 같은 이유)

In [27]:
workclass.reset_index(drop=True)
sample_data1.reset_index(drop=True)
sample_data_workclass = pd.concat([sample_data1,workclass],axis=1)

In [28]:
sample_data_workclass

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,income,occupation,workclass
0,168538,60,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,>50K,Sales,Private
1,101626,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,<=50K,Machine-op-inspct,Private
2,353358,16,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K,Other-service,Private
3,151158,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K,Prof-specialty,Private
4,122234,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K,Adm-clerical,Private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26044,250201,52,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,<=50K,Other-service,Private
26045,238092,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K,Prof-specialty,Private
26046,165694,15,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,<=50K,?,?
26047,151626,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,<=50K,Prof-specialty,Self-emp-not-inc


예측 및 타겟변수 분리 실시

In [29]:
sample_data_null = sample_data_workclass[sample_data_workclass['workclass']=='?']
sample_data_notnull = sample_data_workclass[sample_data_workclass['workclass']!='?']

In [30]:
null_income = sample_data_null['income']
notnull_income = sample_data_notnull['income']
sample_data_null.drop(columns=['income'],inplace=True)
sample_data_notnull.drop(columns=['income'],inplace=True)

In [31]:
null_occupation = sample_data_null['occupation']
notnull_occupation = sample_data_notnull['occupation']
sample_data_null.drop(columns=['occupation'],inplace=True)
sample_data_notnull.drop(columns=['occupation'],inplace=True)

In [32]:
target1 = sample_data_notnull['workclass']
sample_data_null.drop(columns=['workclass'],inplace=True)
sample_data_notnull.drop(columns=['workclass'],inplace=True)

workclass='?' 에 대한 예측 모델 생성

In [33]:
sample_data_null1 = sample_data_null.copy()

In [34]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(sample_data_notnull, target1, 
                                                      test_size=0.3,
                                                      random_state=2020,
                                                      shuffle=True)

In [35]:
x_train_mean = np.mean(x_train[num_columns], axis=0)
x_train_std  = np.std(x_train[num_columns], axis=0)


x_train.loc[:, num_columns] = (x_train[num_columns] - x_train_mean) / x_train_std 
x_valid.loc[:, num_columns] = (x_valid[num_columns] - x_train_mean) / x_train_std
sample_data_null1.loc[:, num_columns] = (sample_data_null1[num_columns] - x_train_mean) / x_train_std

In [36]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [37]:
xgb.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [38]:
workclass_pred = xgb.predict(sample_data_null1)

In [39]:
col = ['workclass']

In [40]:
workclass_pred = pd.DataFrame(workclass_pred,columns=col)

합치기(1)- sample_data_null 과 workclass_pred

In [41]:
workclass_pred = workclass_pred.reset_index(drop=True)
sample_data_null = sample_data_null.reset_index(drop=True)
sample_data_null = pd.concat([sample_data_null, workclass_pred], axis=1)

In [42]:
sample_data_null = sample_data_null.reset_index(drop=True)
null_income = null_income.reset_index(drop=True)
sample_data_null = pd.concat([sample_data_null, null_income], axis=1)

In [43]:
sample_data_null = sample_data_null.reset_index(drop=True)
null_occupation = null_occupation.reset_index(drop=True)
sample_data_null = pd.concat([sample_data_null,null_occupation],axis=1)

In [44]:
sample_data_null

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,income,occupation
0,204226,35,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,?
1,202994,16,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,?
2,505438,30,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,<=50K,?
3,39100,5,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,<=50K,?
4,404601,30,2414,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Self-emp-not-inc,<=50K,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,89852,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,?
1498,321403,40,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,?
1499,214896,40,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,?
1500,165694,15,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Private,<=50K,?


합치기(2) sample_data_notnull

In [45]:
sample_data_notnull.reset_index(drop=True)
target1.reset_index(drop=True)
sample_data_notnull = pd.concat([sample_data_notnull,target1],axis=1)

In [46]:
sample_data_notnull

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,bachelors,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass
0,168538,60,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private
1,101626,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
2,353358,16,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private
3,151158,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private
4,122234,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26042,191389,50,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-not-inc
26043,234960,48,-1887,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-not-inc
26044,250201,52,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Private
26045,238092,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private


income 합치기

In [47]:
sample_data_notnull.reset_index(drop=True)
notnull_income.reset_index(drop=True)
sample_data_notnull = pd.concat([sample_data_notnull,notnull_income],axis=1)

In [48]:
sample_data_notnull

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,income
0,168538,60,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,>50K
1,101626,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K
2,353358,16,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K
3,151158,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K
4,122234,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26042,191389,50,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-not-inc,<=50K
26043,234960,48,-1887,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-not-inc,>50K
26044,250201,52,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K
26045,238092,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K


occupation 합치기

In [49]:
sample_data_notnull.reset_index(drop=True)
notnull_occupation.reset_index(drop=True)
sample_data_notnull = pd.concat([sample_data_notnull,notnull_occupation],axis=1)

In [50]:
sample_data_notnull

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,income,occupation
0,168538,60,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,>50K,Sales
1,101626,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,Machine-op-inspct
2,353358,16,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Other-service
3,151158,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Prof-specialty
4,122234,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Adm-clerical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26042,191389,50,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-not-inc,<=50K,Sales
26043,234960,48,-1887,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-not-inc,>50K,Prof-specialty
26044,250201,52,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Other-service
26045,238092,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Prof-specialty


합치기

In [51]:
sample_data_notnull.reset_index(drop=True)
sample_data_null.reset_index(drop=True)
sample_data = pd.concat([sample_data_notnull,sample_data_null],axis=0)

In [52]:
sample_data

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,income,occupation
0,168538,60,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,>50K,Sales
1,101626,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,Machine-op-inspct
2,353358,16,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Other-service
3,151158,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Prof-specialty
4,122234,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Adm-clerical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,89852,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,?
1498,321403,40,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,?
1499,214896,40,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,?
1500,165694,15,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Private,<=50K,?


test.csv에 대한 전처리

In [53]:
x_test = pd.read_csv(join('data', join('kaggle', 'test.csv')))

In [54]:
x_test['capital'] = x_test['capital_gain'] - x_test['capital_loss']

In [55]:
x_test.drop(columns=['capital_gain','capital_loss'],inplace=True)

In [56]:
#파생변수 생성(age_group)
bins = [0, 29, 39, 64, 120]
labels = ['youth', 'middle', 'middle_elder', 'elder']
x_test['age_group'] = pd.cut(x_test.age, bins, labels = labels,include_lowest = True)

In [57]:
x_test.drop(columns=['age'],inplace=True)

In [58]:
#파생변수 생성(country_group)
condition = [
    x_test['native_country'].str.contains('United-States|Canada', na=False),
    x_test['native_country'].str.contains('Mexico|Cuba|El-Salvador|Guatemala|Columbia|Peru|Nicaragua|Honduras|Ecuador', na=False),
    x_test['native_country'].str.contains('Poland|Germany|France|Ireland|England|Italy|Scotland|Holand-Netherlands', na=False),
    x_test['native_country'].str.contains('Portugal|Hungary|Greece|Yugoslavia', na=False),
    x_test['native_country'].str.contains('Philippines|Vietnam|Thailand|Laos|Cambodia', na=False),
    x_test['native_country'].str.contains('Taiwan|Japan|China|Hong|South', na=False),
    x_test['native_country'].str.contains('Iran|India', na=False),
    x_test['native_country'].str.contains('Puerto-Rico|Haiti|Trinadad&Tobago|Outlying-US(Guam-USVI-etc)', na=False),
    x_test['native_country'].str.contains('Jamaica|Dominican-Republic', na=False)
]

choices = ['N_America', 'S_America', 'High_Europe', 'Mid_Europe', 'ES_Asia', 'E_Asia', 'W_Asia', 'Used_Colony', 'Africa']

x_test['country_group'] = np.select(condition, choices, default = 'United-States')

In [59]:
x_test.drop(columns=['native_country'], inplace=True)

In [60]:
condition = [
    x_test['education'].str.contains('Preschool',na=False),
    x_test['education'].str.contains('1st-4th|5th-6th',na=False),
    x_test['education'].str.contains('7th-8th|9th',na=False),
    x_test['education'].str.contains('10th|11th|12th',na=False),
    x_test['education'].str.contains('HS-grad',na=False),
    x_test['education'].str.contains('Some-college',na=False),
    x_test['education'].str.contains('Assoc-acdm|Assoc-voc',na=False),
    x_test['education'].str.contains('Bachelors',na=False),
    x_test['education'].str.contains('Masters',na=False),
    x_test['education'].str.contains('Prof-school|Doctorate',na=False)
]
choices = ['pre','elementry','middleschool','highschool','highschool_grad','college','assoc','bachelors','ms','ph.d']
x_test['education_level'] = np.select(condition, choices,  default = 'others')


In [61]:
x_test.drop(columns=['education','education_num'], inplace=True)

In [62]:
x_test

,id,workclass,fnlwgt,marital_status,occupation,relationship,race,sex,hours_per_week,capital,age_group,country_group,education_level
0,0,Private,67661,Never-married,Adm-clerical,Other-relative,White,Female,40,0,youth,N_America,college
1,1,Self-emp-inc,37869,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,0,middle_elder,N_America,highschool_grad
2,2,Private,109952,Never-married,Handlers-cleaners,Own-child,White,Male,25,0,youth,N_America,college
3,3,Private,114537,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,0,middle_elder,N_America,assoc
4,4,Private,51264,Married-civ-spouse,Prof-specialty,Husband,White,Male,99,0,middle,High_Europe,ph.d
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,6507,Private,61343,Married-civ-spouse,Sales,Husband,White,Male,40,0,middle,N_America,bachelors
6508,6508,Self-emp-inc,32185,Married-civ-spouse,Tech-support,Husband,White,Male,40,0,middle_elder,N_America,bachelors
6509,6509,Private,409189,Married-civ-spouse,Other-service,Husband,White,Male,40,0,middle,S_America,elementry
6510,6510,Private,180342,Married-civ-spouse,Craft-repair,Husband,White,Male,40,0,middle,N_America,highschool_grad


In [63]:
#변수 분리

In [64]:
workclass=x_test['workclass']
occupation = x_test['occupation']
id = x_test['id']
x_test.drop(columns=['workclass','occupation','id'],inplace=True)

In [65]:
#연속형 범주형 구분

In [66]:
cat_columns = [c for (c, t) in zip(x_test.dtypes.index, x_test.dtypes) if t == 'O'] 
num_columns = [c for c in x_test.columns if c not in cat_columns]

In [67]:
x_test['age_group'] = x_test['age_group'].astype('object')

In [68]:
cat_columns = [c for (c, t) in zip(x_test.dtypes.index, x_test.dtypes) if t == 'O'] 
num_columns = [c for c in x_test.columns if c not in cat_columns]

In [69]:
x_test_num = x_test[num_columns]
x_test_cat = x_test[cat_columns]

In [70]:
#범주형 인코딩

In [71]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
x_test_cat = ohe.fit_transform(x_test_cat)

In [72]:
ohe_columns=ohe.categories_[0].tolist()
for i in range(len(ohe.categories_)-1):
        ohe_columns += ohe.categories_[i+1].tolist()

In [73]:
x_test_cat = pd.DataFrame(x_test_cat,columns = ohe_columns)

In [74]:
#num col 합치기

In [75]:
x_test_num.reset_index(drop=True)
x_test_cat.reset_index(drop=True)
x_test = pd.concat([x_test_num,x_test_cat],axis=1)

In [76]:
x_test

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,assoc,bachelors,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre
0,67661,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,37869,50,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,109952,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,114537,50,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,51264,99,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,61343,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6508,32185,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6509,409189,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6510,180342,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [77]:
#workclass ocupation 붙이기

In [78]:
x_test = x_test.reset_index(drop=True)
workclass= workclass.reset_index(drop=True)
x_test = pd.concat([x_test,workclass],axis=1)

In [79]:
x_test.reset_index(drop=True)
occupation.reset_index(drop=True)
x_test = pd.concat([x_test,occupation],axis=1)

In [80]:
x_test.reset_index(drop=True)
id.reset_index(drop=True)
x_test = pd.concat([x_test,id],axis=1)

In [81]:
# original data
x_test

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation,id
0,67661,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Adm-clerical,0
1,37869,50,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Self-emp-inc,Exec-managerial,1
2,109952,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Handlers-cleaners,2
3,114537,50,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Exec-managerial,3
4,51264,99,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Private,Prof-specialty,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,61343,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Sales,6507
6508,32185,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-inc,Tech-support,6508
6509,409189,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Other-service,6509
6510,180342,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,Craft-repair,6510


In [82]:
#occupation 분리

In [83]:
x_test_null = x_test[x_test['occupation']=='?']
x_test_notnull = x_test[x_test['occupation'] !='?']

In [84]:
x_test_null

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation,id
11,216256,30,3464,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,?,?,11
13,163665,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,?,?,13
15,27184,38,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,?,?,15
29,131982,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,?,?,29
32,132737,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,?,?,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6377,181265,30,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,?,?,6377
6393,385901,22,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,?,?,6393
6422,275943,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,?,?,6422
6444,163998,20,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,?,?,6444


In [85]:
x_test_notnull

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation,id
0,67661,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Adm-clerical,0
1,37869,50,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Self-emp-inc,Exec-managerial,1
2,109952,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Handlers-cleaners,2
3,114537,50,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Exec-managerial,3
4,51264,99,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Private,Prof-specialty,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,61343,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Sales,6507
6508,32185,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-inc,Tech-support,6508
6509,409189,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Other-service,6509
6510,180342,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,Craft-repair,6510


In [86]:
workclass_null = x_test_null['workclass']
occupation_null = x_test_null['occupation']
id_null =x_test_null['id']
x_test_null.drop(columns=['workclass','occupation','id'],inplace=True)

In [87]:
#test 내용이 train에 섞이지 않도록 주의
predict = xgb.predict(x_test_null)

In [88]:
col = ['workclass']

In [89]:
predict = pd.DataFrame(predict,columns=col)

In [90]:
x_test_null = x_test_null.reset_index(drop=True)
predict = predict.reset_index(drop=True)
x_test_null = pd.concat([x_test_null,predict],axis=1)

In [91]:
x_test_null

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,bachelors,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass
0,216256,30,3464,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private
1,163665,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private
2,27184,38,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private
3,131982,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private
4,132737,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,181265,30,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private
331,385901,22,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private
332,275943,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private
333,163998,20,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private


In [92]:
#occupation 붙이기

In [93]:
occupation_null = occupation_null.reset_index(drop=True)
x_test_null = x_test_null.reset_index(drop=True)
x_test_null = pd.concat([x_test_null, occupation_null],axis=1)

In [94]:
x_test_null

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation
0,216256,30,3464,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,?
1,163665,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,?
2,27184,38,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,?
3,131982,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,?
4,132737,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Private,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,181265,30,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,?
331,385901,22,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,?
332,275943,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,?
333,163998,20,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,?


In [95]:
#id 붙이기

In [96]:
id_null = id_null.reset_index(drop=True)
x_test_null = x_test_null.reset_index(drop=True)
x_test_null = pd.concat([x_test_null,id_null],axis=1)

In [97]:
#null notnull 합치기

In [98]:
x_test_null = x_test_null.reset_index(drop=True)
x_test_notnull = x_test_notnull.reset_index(drop=True)
x_test = pd.concat([x_test_null,x_test_notnull],axis=0)

In [99]:
x_test

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation,id
0,216256,30,3464,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,?,11
1,163665,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,?,13
2,27184,38,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,?,15
3,131982,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,?,29
4,132737,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Private,?,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6172,61343,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Sales,6507
6173,32185,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-inc,Tech-support,6508
6174,409189,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Other-service,6509
6175,180342,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,Craft-repair,6510


occupation 모델 만들기

In [100]:
sample_data

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,income,occupation
0,168538,60,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,>50K,Sales
1,101626,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,Machine-op-inspct
2,353358,16,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Other-service
3,151158,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Prof-specialty
4,122234,20,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,Adm-clerical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,89852,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,?
1498,321403,40,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,?
1499,214896,40,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,?
1500,165694,15,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Private,<=50K,?


#occupation ?  구분

In [101]:
sample_data_null = sample_data[sample_data['occupation']=='?']
sample_data_notnull = sample_data[sample_data['occupation'] !='?']

In [102]:
sample_data_null

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,income,occupation
7929,237272,30,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,<=50K,?
8698,157131,10,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,<=50K,?
13008,153663,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Never-worked,<=50K,?
15076,462294,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Never-worked,<=50K,?
17531,206359,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,<=50K,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,89852,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,<=50K,?
1498,321403,40,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,?
1499,214896,40,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,<=50K,?
1500,165694,15,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Private,<=50K,?


#변수 분리 및 제거

In [103]:
workclass_null = sample_data_null['workclass']
income_null = sample_data_null['income']
sample_data_null.drop(columns=['workclass','income','occupation'],inplace=True)

In [104]:
workclass_notnull = sample_data_notnull['workclass']
income_notnull = sample_data_notnull['income']
target = sample_data_notnull['occupation']
sample_data_notnull.drop(columns=['workclass','income','occupation'],inplace=True)

#occupation 예측 모델 생성

In [105]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(sample_data_notnull, target, 
                                                      test_size=0.3,
                                                      random_state=2020,
                                                      shuffle=True)

In [106]:
sample_data_null1 = sample_data_null.copy()

In [107]:
x_train_mean = np.mean(x_train[num_columns], axis=0)
x_train_std  = np.std(x_train[num_columns], axis=0)


x_train.loc[:, num_columns] = (x_train[num_columns] - x_train_mean) / x_train_std 
x_valid.loc[:, num_columns] = (x_valid[num_columns] - x_train_mean) / x_train_std
sample_data_null1.loc[:, num_columns] = (sample_data_null1[num_columns] - x_train_mean) / x_train_std

In [108]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [109]:
xgb.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [110]:
occupation_pred = xgb.predict(sample_data_null1)

In [111]:
col = ['occupation']

In [112]:
occupation_pred = pd.DataFrame(occupation_pred,columns=col)

#workclass 합치기

In [113]:
sample_data_null = sample_data_null.reset_index(drop=True)
workclass_null = workclass_null.reset_index(drop=True)
sample_data_null = pd.concat([sample_data_null,workclass_null],axis=1)

#합치기(1)- sample_data_null 과 occupation_pred

In [114]:
occupation_pred = occupation_pred.reset_index(drop=True)
sample_data_null = sample_data_null.reset_index(drop=True)
sample_data_null = pd.concat([sample_data_null,occupation_pred],axis=1)

In [115]:
sample_data_null

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation
0,237272,30,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Other-service
1,157131,10,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Sales
2,153663,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Prof-specialty
3,462294,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Adm-clerical
4,206359,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Craft-repair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,89852,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Farming-fishing
1504,321403,40,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,Craft-repair
1505,214896,40,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,Machine-op-inspct
1506,165694,15,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Private,Prof-specialty


#income 합치기

In [116]:
sample_data_null = sample_data_null.reset_index(drop=True)
income_null = income_null.reset_index(drop=True)
sample_data_null = pd.concat([sample_data_null, income_null], axis=1)

In [117]:
sample_data_null

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation,income
0,237272,30,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Other-service,<=50K
1,157131,10,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Sales,<=50K
2,153663,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Prof-specialty,<=50K
3,462294,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Adm-clerical,<=50K
4,206359,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Craft-repair,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,89852,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Farming-fishing,<=50K
1504,321403,40,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,Craft-repair,<=50K
1505,214896,40,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private,Machine-op-inspct,<=50K
1506,165694,15,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Private,Prof-specialty,<=50K


sample_data_notnull 붙이기

In [118]:
sample_data_notnull.reset_index(drop=True)
workclass_notnull.reset_index(drop=True)
sample_data_notnull = pd.concat([sample_data_notnull,workclass_notnull],axis=1)

In [119]:
sample_data_notnull.reset_index(drop=True)
target.reset_index(drop=True)
sample_data_notnull = pd.concat([sample_data_notnull,target],axis=1)

In [120]:
sample_data_notnull.reset_index(drop=True)
income_notnull.reset_index(drop=True)
sample_data_notnull = pd.concat([sample_data_notnull,income_notnull],axis=1)

#sample_data_notnull sample_data_null 합치기

In [121]:
sample_data_notnull = sample_data_notnull.reset_index(drop=True)
sample_data_null = sample_data_null.reset_index(drop=True)
sample_data = pd.concat([sample_data_null,sample_data_notnull],axis=0)

In [122]:
sample_data

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation,income
0,237272,30,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Other-service,<=50K
1,157131,10,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Sales,<=50K
2,153663,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Prof-specialty,<=50K
3,462294,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Adm-clerical,<=50K
4,206359,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Never-worked,Craft-repair,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24536,191389,50,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-not-inc,Sales,<=50K
24537,234960,48,-1887,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-not-inc,Prof-specialty,>50K
24538,250201,52,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Private,Other-service,<=50K
24539,238092,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Prof-specialty,<=50K


#workclass occupation 인코딩 처리

In [123]:
encoding = sample_data[['workclass','occupation']]
sample_data.drop(columns=['workclass','occupation'],inplace=True)

In [124]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
encoding = ohe.fit_transform(encoding)

In [125]:
ohe_columns=ohe.categories_[0].tolist()
for i in range(len(ohe.categories_)-1):
        ohe_columns += ohe.categories_[i+1].tolist()

In [126]:
encoding = pd.DataFrame(encoding,columns = ohe_columns)

In [127]:
encoding

,Federal-gov,Local-gov,Never-worked,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,Adm-clerical,Armed-Forces,...,Farming-fishing,Handlers-cleaners,Machine-op-inspct,Other-service,Priv-house-serv,Prof-specialty,Protective-serv,Sales,Tech-support,Transport-moving
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26044,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
26045,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
26046,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
26047,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [128]:
encoding = encoding.reset_index(drop=True)
sample_data =sample_data.reset_index(drop=True)
sample_data=pd.concat([encoding,sample_data],axis=1)

In [129]:
pd.isna(sample_data).sum()

Federal-gov     0
Local-gov       0
Never-worked    0
Private         0
Self-emp-inc    0
               ..
middleschool    0
ms              0
ph.d            0
pre             0
income          0
Length: 70, dtype: int64

In [130]:
sample_data

,Federal-gov,Local-gov,Never-worked,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,Adm-clerical,Armed-Forces,...,bachelors,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,income
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,<=50K
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,<=50K
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26044,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K
26045,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,>50K
26046,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,<=50K
26047,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K


In [131]:
x_test_notnull

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation,id
0,67661,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Adm-clerical,0
1,37869,50,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Self-emp-inc,Exec-managerial,1
2,109952,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Handlers-cleaners,2
3,114537,50,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Exec-managerial,3
4,51264,99,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Private,Prof-specialty,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6172,61343,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Sales,6507
6173,32185,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-inc,Tech-support,6508
6174,409189,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Other-service,6509
6175,180342,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,Craft-repair,6510


#x_test에 대해 occpuation 결측치 채워넣기

In [132]:
pd.isna(x_test).sum()

fnlwgt                   0
hours_per_week           0
capital                  0
Divorced                 0
Married-AF-spouse        0
Married-civ-spouse       0
Married-spouse-absent    0
Never-married            0
Separated                0
Widowed                  0
Husband                  0
Not-in-family            0
Other-relative           0
Own-child                0
Unmarried                0
Wife                     0
Amer-Indian-Eskimo       0
Asian-Pac-Islander       0
Black                    0
Other                    0
White                    0
Female                   0
Male                     0
elder                    0
middle                   0
middle_elder             0
youth                    0
Africa                   0
ES_Asia                  0
E_Asia                   0
High_Europe              0
Mid_Europe               0
N_America                0
S_America                0
United-States            0
Used_Colony              0
W_Asia                   0
a

#occupation ? 분리

In [133]:
x_test_null = x_test[x_test['occupation']=='?']
x_test_notnull = x_test[x_test['occupation'] !='?']

In [134]:
workclass_null = x_test_null['workclass']
occupation_null = x_test_null['occupation']
id_null = x_test_null['id']

x_test_null.drop(columns=['workclass','occupation','id'],inplace=True)

In [135]:
x_test_null

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,assoc,bachelors,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre
0,216256,30,3464,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,163665,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,27184,38,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,131982,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,132737,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,181265,30,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
331,385901,22,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
332,275943,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
333,163998,20,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


#predict

In [136]:
predict = xgb.predict(x_test_null)

In [137]:
col = ['occupation']

In [138]:
occupation = pd.DataFrame(predict,columns=col)

In [139]:
occupation

,occupation
0,Farming-fishing
1,Other-service
2,Other-service
3,Other-service
4,Other-service
...,...
330,Other-service
331,Other-service
332,Farming-fishing
333,Farming-fishing


In [140]:
x_test_notnull

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation,id
0,67661,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Adm-clerical,0
1,37869,50,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Self-emp-inc,Exec-managerial,1
2,109952,25,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Handlers-cleaners,2
3,114537,50,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Exec-managerial,3
4,51264,99,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Private,Prof-specialty,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6172,61343,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Sales,6507
6173,32185,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-inc,Tech-support,6508
6174,409189,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Other-service,6509
6175,180342,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,Craft-repair,6510


In [141]:
x_test_null = x_test_null.reset_index(drop=True)
workclass_null = workclass_null.reset_index(drop=True)
x_test_null = pd.concat([x_test_null,workclass_null],axis=1)

In [142]:
x_test_null = x_test_null.reset_index(drop=True)
occupation = occupation.reset_index(drop=True)
x_test_null = pd.concat([x_test_null,occupation],axis=1)

In [143]:
x_test_null = x_test_null.reset_index(drop=True)
id_null = id_null.reset_index(drop=True)
x_test_null = pd.concat([x_test_null,id_null],axis=1)

In [144]:
x_test_null= x_test_null.reset_index(drop=True)
x_test_notnull = x_test_notnull.reset_index(drop=True)
x_test = pd.concat([x_test_null,x_test_notnull],axis=0)

In [145]:
x_test

,fnlwgt,hours_per_week,capital,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed,...,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre,workclass,occupation,id
0,216256,30,3464,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,Farming-fishing,11
1,163665,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Other-service,13
2,27184,38,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,Other-service,15
3,131982,40,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Other-service,29
4,132737,4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Private,Other-service,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6172,61343,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Sales,6507
6173,32185,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Self-emp-inc,Tech-support,6508
6174,409189,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Private,Other-service,6509
6175,180342,40,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private,Craft-repair,6510


#workplace occupation 에 대해 인코딩 실시

In [146]:
encoding = x_test[['workclass','occupation']]
x_test.drop(columns=['workclass','occupation'],inplace=True)

In [147]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
encoding = ohe.fit_transform(encoding)

In [148]:
ohe_columns=ohe.categories_[0].tolist()
for i in range(len(ohe.categories_)-1):
        ohe_columns += ohe.categories_[i+1].tolist()

In [149]:
encoding = pd.DataFrame(encoding,columns = ohe_columns)

In [150]:
encoding = encoding.reset_index(drop=True)
x_test =x_test.reset_index(drop=True)
x_test=pd.concat([encoding,x_test],axis=1)

In [151]:
x_test.columns

Index(['Federal-gov', 'Local-gov', 'Private', 'Self-emp-inc',
       'Self-emp-not-inc', 'State-gov', 'Without-pay', 'Adm-clerical',
       'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing',
       'Handlers-cleaners', 'Machine-op-inspct', 'Other-service',
       'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales',
       'Tech-support', 'Transport-moving', 'fnlwgt', 'hours_per_week',
       'capital', 'Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
       'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed',
       'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried',
       'Wife', 'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other',
       'White', 'Female', 'Male', 'elder', 'middle', 'middle_elder', 'youth',
       'Africa', 'ES_Asia', 'E_Asia', 'High_Europe', 'Mid_Europe', 'N_America',
       'S_America', 'United-States', 'Used_Colony', 'W_Asia', 'assoc',
       'bachelors', 'college', 'elementry', 'highs

In [152]:
sample_data.columns

Index(['Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc',
       'Self-emp-not-inc', 'State-gov', 'Without-pay', 'Adm-clerical',
       'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing',
       'Handlers-cleaners', 'Machine-op-inspct', 'Other-service',
       'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales',
       'Tech-support', 'Transport-moving', 'fnlwgt', 'hours_per_week',
       'capital', 'Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
       'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed',
       'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried',
       'Wife', 'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other',
       'White', 'Female', 'Male', 'elder', 'middle', 'middle_elder', 'youth',
       'Africa', 'ES_Asia', 'E_Asia', 'High_Europe', 'Mid_Europe', 'N_America',
       'S_America', 'United-States', 'Used_Colony', 'W_Asia', 'assoc',
       'bachelors', 'college', 'el

In [153]:
num_columns

['fnlwgt', 'hours_per_week', 'capital']

#전처리 완료

In [154]:
x_test.sort_values(by='id',inplace=True)

In [155]:
x_test.drop(columns='id',inplace=True)

In [156]:
x_test['Never-worked'] = 0

In [157]:
label = sample_data['income']

In [158]:
sample_data.drop(columns='income',inplace=True)

In [159]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(label)

In [160]:
sample_data.columns

Index(['Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc',
       'Self-emp-not-inc', 'State-gov', 'Without-pay', 'Adm-clerical',
       'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing',
       'Handlers-cleaners', 'Machine-op-inspct', 'Other-service',
       'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales',
       'Tech-support', 'Transport-moving', 'fnlwgt', 'hours_per_week',
       'capital', 'Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
       'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed',
       'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried',
       'Wife', 'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other',
       'White', 'Female', 'Male', 'elder', 'middle', 'middle_elder', 'youth',
       'Africa', 'ES_Asia', 'E_Asia', 'High_Europe', 'Mid_Europe', 'N_America',
       'S_America', 'United-States', 'Used_Colony', 'W_Asia', 'assoc',
       'bachelors', 'college', 'el

In [161]:
x_test = x_test[['Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc',
       'Self-emp-not-inc', 'State-gov', 'Without-pay', 'Adm-clerical',
       'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing',
       'Handlers-cleaners', 'Machine-op-inspct', 'Other-service',
       'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales',
       'Tech-support', 'Transport-moving', 'fnlwgt', 'hours_per_week',
       'capital', 'Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
       'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed',
       'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried',
       'Wife', 'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other',
       'White', 'Female', 'Male', 'elder', 'middle', 'middle_elder', 'youth',
       'Africa', 'ES_Asia', 'E_Asia', 'High_Europe', 'Mid_Europe', 'N_America',
       'S_America', 'United-States', 'Used_Colony', 'W_Asia', 'assoc',
       'bachelors', 'college', 'elementry', 'highschool', 'highschool_grad',
       'middleschool', 'ms', 'ph.d', 'pre']]

In [162]:
x_test

,Federal-gov,Local-gov,Never-worked,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,Adm-clerical,Armed-Forces,...,assoc,bachelors,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre
335,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
336,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
337,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
338,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
339,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6508,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6509,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6510,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [163]:
sample_data

,Federal-gov,Local-gov,Never-worked,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,Adm-clerical,Armed-Forces,...,assoc,bachelors,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26044,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26045,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26046,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
26047,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [164]:
sample_data[num_columns]

,fnlwgt,hours_per_week,capital
0,237272,30,0
1,157131,10,0
2,153663,4,0
3,462294,40,0
4,206359,40,0
...,...,...,...
26044,191389,50,0
26045,234960,48,-1887
26046,250201,52,0
26047,238092,40,0


#test(only split)

In [165]:
sample_data

,Federal-gov,Local-gov,Never-worked,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,Adm-clerical,Armed-Forces,...,assoc,bachelors,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26044,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26045,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26046,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
26047,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [166]:
x_test

,Federal-gov,Local-gov,Never-worked,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,Adm-clerical,Armed-Forces,...,assoc,bachelors,college,elementry,highschool,highschool_grad,middleschool,ms,ph.d,pre
335,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
336,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
337,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
338,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
339,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6508,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6509,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6510,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


간단한 모델

In [167]:
from sklearn.model_selection import train_test_split

tmp_train, tmp_valid, y_train, y_valid = train_test_split(sample_data, label, 
                                                          test_size=0.3,
                                                          random_state=2020,
                                                          shuffle=True,
                                                          stratify=label)

In [168]:
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
lgb = LGBMClassifier(tree_method='gpu_hist')

lgb.fit(tmp_train, y_train)

y_pred = lgb.predict(tmp_valid)

print(f"LightGBM F1 Score: {f1_score(y_valid, y_pred, average='micro')}")

[LightGBM] [Warning] Unknown parameter: tree_method
LightGBM F1 Score: 0.8726807421625079


OOF ENSEMBLE 

전처리 프로세스(정규화)

In [171]:
def preprocess(x_train,x_valid,x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test = x_test.copy()
    
    
    
        
    x_train_mean = np.mean(tmp_x_train[num_columns], axis=0)
    x_train_std  = np.std(tmp_x_train[num_columns], axis=0)

    tmp_x_train.loc[:, num_columns] = (tmp_x_train[num_columns] - x_train_mean) / (x_train_std + 1e-4)
    tmp_x_valid.loc[:, num_columns] = (tmp_x_valid[num_columns] - x_train_mean) / (x_train_std + 1e-4)
    tmp_x_test.loc[:, num_columns] = (tmp_x_test[num_columns] - x_train_mean) / (x_train_std + 1e-4)
    
    
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

StratifiedKfold 실시

In [175]:
from sklearn.model_selection import StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)



In [176]:
#stackoverflow 참고
def xgb_f1(y, t, threshold=0.5):
    t = t.get_label()
    y_bin = (y > threshold).astype(int) 
    return 'f1', f1_score(t, y_bin, average='micro')

In [177]:
val_scores = list()
oof_pred = np.zeros((x_test.shape[0], ))

for i, (trn_idx, val_idx) in enumerate(skf.split(sample_data, label)):
    x_train, y_train = sample_data.iloc[trn_idx, :], label[trn_idx]
    x_valid, y_valid = sample_data.iloc[val_idx, :], label[val_idx]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, x_test)
    
    
    
    # 모델생성
    clf = XGBClassifier(n_estimators=100,
                        random_state=2020,
                        tree_method='gpu_hist',
                        n_jobs=2)
    
    # 모델학습
    clf.fit(x_train, y_train,
            eval_set = [[x_valid, y_valid]], 
            eval_metric = xgb_f1,        
            early_stopping_rounds = 100,
            verbose = 100,  )

    # 훈련, 검증 데이터 F1 Score 확인
    trn_f1_score = f1_score(y_train, clf.predict(x_train))
    val_f1_score = f1_score(y_valid, clf.predict(x_valid))
    print('{} Fold, train f1_score : {:.4f}4, validation f1_score : {:.4f}\n'.format(i, trn_f1_score, val_f1_score))
    
    val_scores.append(val_f1_score)
    
    oof_pred += clf.predict_proba(x_test)[: , 1] / n_splits
    

# 교차 검증 F1 Score 평균 계산
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

[0]	validation_0-error:0.14875	validation_0-f1:0.81785
Multiple eval metrics have been passed: 'validation_0-f1' will be used for early stopping.

Will train until validation_0-f1 hasn't improved in 100 rounds.
[99]	validation_0-error:0.13570	validation_0-f1:0.86296
0 Fold, train f1_score : 0.65304, validation f1_score : 0.6308

[0]	validation_0-error:0.15355	validation_0-f1:0.82553
Multiple eval metrics have been passed: 'validation_0-f1' will be used for early stopping.

Will train until validation_0-f1 hasn't improved in 100 rounds.
[99]	validation_0-error:0.12918	validation_0-f1:0.86871
1 Fold, train f1_score : 0.67394, validation f1_score : 0.6633

[0]	validation_0-error:0.14434	validation_0-f1:0.82111
Multiple eval metrics have been passed: 'validation_0-f1' will be used for early stopping.

Will train until validation_0-f1 hasn't improved in 100 rounds.
[99]	validation_0-error:0.12841	validation_0-f1:0.86584
2 Fold, train f1_score : 0.66704, validation f1_score : 0.6773

[0]	val

In [184]:
(oof_pred>0.5).astype(int)

array([0, 0, 0, ..., 0, 0, 0])